# Customer Retention Analysis and RFM Segmentation in E-commerce

## Dataset Overview & Intent
- **Source:** Kaggle (E-commerce Purchase History from Electronics Store), provider: REES46 Marketing Platform  
- **Scale:** ~2.6M purchase events; multiple items per `order_id`  
- **Goal:** Understand repeat purchasing behavior and customer value concentration; build RFM-based segments for actionable retention strategies  

## Objectives
1) Quantify customer retention patterns across time using cohort analysis  
2) Identify high-value customers through RFM (Recency, Frequency, Monetary) analytics  
3) Segment customers into actionable groups to inform targeted retention and marketing strategies  

## Import Required Libraries


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

## Load Dataset

In [2]:
DATA_PATH = "../data/kz.csv"  
df = pd.read_csv(DATA_PATH, low_memory=False)

## Quick Preview

In [3]:
df.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


## Dataset Info

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2633521 entries, 0 to 2633520
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   order_id       int64  
 2   product_id     int64  
 3   category_id    float64
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        float64
dtypes: float64(3), int64(2), object(3)
memory usage: 160.7+ MB


**Observation:** `event_time` is `object`; all identifiers are numeric (`order_id`, `product_id` as int; `user_id`, `category_id` as float). IDs should be strings to prevent precision loss/scientific notation. `event_time` should be parsed as datetime (UTC).

## Load the dataset with Explicit Dtypes and Parsed Timestamps


In [5]:
dtype_map = {
    "order_id": "string",
    "user_id": "string",
    "product_id": "string",
    "category_id": "string",
    "category_code": "string",
    "brand": "string",
    "price": "float32",
}

df = pd.read_csv(
    DATA_PATH,
    dtype=dtype_map,
    parse_dates=["event_time"],
    low_memory=False
)

df["event_time"] = pd.to_datetime(df["event_time"], utc=True, errors="coerce")

## Dataset Info After Schema Definition

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2633521 entries, 0 to 2633520
Data columns (total 8 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   order_id       string             
 2   product_id     string             
 3   category_id    string             
 4   category_code  string             
 5   brand          string             
 6   price          float32            
 7   user_id        string             
dtypes: datetime64[ns, UTC](1), float32(1), string(6)
memory usage: 150.7 MB


## Dataset Dimensions

In [7]:
df.shape

(2633521, 8)

## Date Coverage

In [8]:
df["event_time"].min(), df["event_time"].max()

(Timestamp('1970-01-01 00:33:40+0000', tz='UTC'),
 Timestamp('2020-11-21 10:10:30+0000', tz='UTC'))

**Observation:** Minimum timestamp shows 1970-01-01, indicating invalid entries

## Validate Event Time

In [9]:
before_2020 = (df["event_time"] < "2020-01-01").sum()
nat_count   = df["event_time"].isna().sum()
print("Rows before 2020-01-01:", before_2020)
print("Rows with NaT:", nat_count)

df.loc[df["event_time"] < "2020-01-01", "event_time"].head()

Rows before 2020-01-01: 19631
Rows with NaT: 0


28813   1970-01-01 00:33:40+00:00
28814   1970-01-01 00:33:40+00:00
28815   1970-01-01 00:33:40+00:00
28816   1970-01-01 00:33:40+00:00
28817   1970-01-01 00:33:40+00:00
Name: event_time, dtype: datetime64[ns, UTC]

## Filter event_time to 2020

In [11]:
total_before = len(df)
mask_valid = (df["event_time"] >= "2020-01-01") & (df["event_time"] <= "2020-12-31")
removed = (~mask_valid).sum()
df = df.loc[mask_valid].reset_index(drop=True)

print("Removed rows:", removed, f"({removed/total_before:.2%})")
print("Rows after clean:", len(df))

Removed rows: 19631 (0.75%)
Rows after clean: 2613890


## Date coverage after filtering

In [12]:
df["event_time"].min(), df["event_time"].max()

(Timestamp('2020-01-05 03:26:46+0000', tz='UTC'),
 Timestamp('2020-11-21 10:10:30+0000', tz='UTC'))

## Missing values

In [14]:
na_pct = (
    df.isna()
      .mean()
      .mul(100)
      .round(2)
      .sort_values(ascending=False)
)
na_pct

user_id          78.47
category_code    23.23
brand            19.18
category_id      16.37
price            16.37
event_time        0.00
order_id          0.00
product_id        0.00
dtype: float64

**Observation:** The dataset shows substantial missingness, with user_id absent in 78.47% of rows. Notable gaps are also present in category_code (23.23%), brand (19.18%), and both category_id and price (16.37% each). In contrast, event_time is fully complete, ensuring reliable temporal analysis.

## Assess user_id availability

In [17]:
rows_total = len(df)
rows_with_uid = df["user_id"].notna().sum()
unique_users  = df.loc[df["user_id"].notna(), "user_id"].nunique()
orders_with_uid = df.loc[df["user_id"].notna(), "order_id"].nunique()

print("Rows with user_id:", rows_with_uid, f"({rows_with_uid/rows_total:.2%})")
print("Unique users:", unique_users)
print("Orders with user_id:", orders_with_uid)

Rows with user_id: 562862 (21.53%)
Unique users: 233575
Orders with user_id: 399750


## Subset transactions with valid user_id

In [9]:
df_uid = df[df["user_id"].notna()].copy()
len(df_uid), df_uid["user_id"].nunique(), df_uid["order_id"].nunique()

(564169, 233834, 400538)

**Observation:** After restricting to transactions with valid user_id, the dataset contains 564,169 rows, representing 233,834 unique customers across 400,538 orders.


## Price sanity check (user_id subset)

In [10]:
null_price   = df_uid["price"].isna().sum()
non_positive = (df_uid["price"] <= 0).sum()
summary      = df_uid["price"].describe(percentiles=[.5, .75, .9, .95, .99])

print("Null price:", null_price)
print("Non-positive price (<=0):", non_positive)
summary

Null price: 0
Non-positive price (<=0): 39


count    564169.000000
mean        208.269287
std         304.536530
min           0.000000
50%          87.940002
75%         277.750000
90%         543.960022
95%         856.229980
99%        1385.390015
max       18328.679688
Name: price, dtype: float64

**Observation:** (revised): Prices have no nulls and 39 non-positive values; the upper tail is large relative to the median (P99 ≈ 1,385 vs. median ≈ 88; max ≈ 18,329).

## Remove invalid prices (user_id subset)

In [11]:
before = len(df_uid)
df_uid = df_uid[df_uid["price"] > 0].reset_index(drop=True)
removed = before - len(df_uid)
print("Removed non-positive prices:", removed, f"({removed/before:.4%})")

Removed non-positive prices: 39 (0.0069%)


Non-positive prices were negligible (39 rows; 0.0069%) and have been removed.

## Standardize brand and category_code

In [14]:
df_uid["brand"] = df_uid["brand"].str.lower().str.strip()
df_uid["category_code"] = df_uid["category_code"].str.lower().str.strip()

## Backfill category_code from category_id

In [15]:
cat_map = (
    df_uid.dropna(subset=["category_id","category_code"])
          .groupby("category_id")["category_code"]
          .agg(lambda s: s.value_counts().idxmax())
)

df_uid["category_code"] = df_uid["category_code"].fillna(df_uid["category_id"].map(cat_map))
df_uid["category_code"] = df_uid["category_code"].fillna("unknown")
df_uid["brand"] = df_uid["brand"].fillna("unknown")

## Missingness after fills (user_id subset)

In [16]:
df_uid[["brand","category_id","category_code"]].isna().mean().mul(100).round(2)

brand            0.0
category_id      0.0
category_code    0.0
dtype: float64

## Validate order -> user consistency 

In [17]:
# Each order_id should belong to exactly one user_id
users_per_order = df_uid.groupby("order_id")["user_id"].nunique()
inconsistent_orders = (users_per_order > 1).sum()
print("Orders with >1 user_id:", inconsistent_orders)

Orders with >1 user_id: 0


## Build orders table

In [18]:
# Helper to pick the most frequent user_id per order (in case of anomalies)
def most_frequent(s):
    return s.value_counts().idxmax()

orders = (
    df_uid.groupby("order_id").agg(
        user_id=("user_id", most_frequent),
        order_dt=("event_time", "max"),
        order_revenue=("price", "sum"),
        items=("product_id", "count"),
        distinct_products=("product_id", "nunique"),
        distinct_categories=("category_code", "nunique"),
        distinct_brands=("brand", "nunique")
    )
    .reset_index()
)

orders.shape, orders.head(3)

((400516, 8),
               order_id              user_id                  order_dt  order_revenue  items  distinct_products  \
 0  2294359932054536986  1515915625441993984 2020-04-24 11:50:39+00:00     324.019989      2                  1   
 1  2294444024058086220  1515915625447879434 2020-04-24 14:37:43+00:00     155.039993      2                  1   
 2  2294584263154074236  1515915625443148002 2020-04-24 19:16:21+00:00     217.570007      1                  1   
 
    distinct_categories  distinct_brands  
 0                    1                1  
 1                    1                1  
 2                    1                1  )

## Quick sanity on orders

In [19]:
print("Zero/negative revenue orders:", (orders["order_revenue"] <= 0).sum())
orders["order_revenue"].describe(percentiles=[.5,.75,.9,.95,.99])

Zero/negative revenue orders: 0


count    400516.000000
mean        293.369263
std         397.076935
min           0.020000
50%         148.339996
75%         393.500000
90%         810.159973
95%        1041.640015
99%        1712.920044
max       18699.029297
Name: order_revenue, dtype: float64

## Order revenue tail check

In [20]:
q = orders["order_revenue"].quantile([0.95, 0.99, 0.995, 0.999])
tail_counts = {
    "above_p99": (orders["order_revenue"] > q.loc[0.99]).sum(),
    "above_p995": (orders["order_revenue"] > q.loc[0.995]).sum(),
    "above_p999": (orders["order_revenue"] > q.loc[0.999]).sum(),
}
q, tail_counts

(0.950    1041.640015
 0.990    1712.920044
 0.995    2148.770105
 0.999    3473.019691
 Name: order_revenue, dtype: float64,
 {'above_p99': 3990, 'above_p995': 2003, 'above_p999': 401})

## Winsorize order_revenue at P99.5

In [21]:
cap = orders["order_revenue"].quantile(0.995)
orders["order_revenue_cap"] = orders["order_revenue"].clip(upper=cap)

orders[["order_revenue","order_revenue_cap"]].describe(percentiles=[.5,.75,.9,.95,.99])

,order_revenue,order_revenue_cap
count,400516.000000,400516.000000
mean,293.369263,289.277483
std,397.076935,365.851803
min,0.020000,0.020000
50%,148.339996,148.339996
75%,393.500000,393.500000
90%,810.159973,810.159973
95%,1041.640015,1041.640015
99%,1712.920044,1712.920044
max,18699.029297,2148.770105
